# Домашнее задание 5. Анализ данных на Spark SQL

In [96]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *

## 1. Создать таблицу, используя csv-файл

In [ ]:
spark = (
    SparkSession.builder
    .appName("my_spark_db")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", "./spark-warehouse")
    .enableHiveSupport()
    .getOrCreate()
)

In [20]:
df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("raw_data/Hotel.csv")
)

In [21]:
spark.sql("CREATE DATABASE IF NOT EXISTS default_db")
spark.sql("USE default_db")

df.write \
  .mode("overwrite") \
  .format("parquet") \
  .saveAsTable("logs_hotel")

In [22]:
spark.sql("SELECT * FROM logs_hotel LIMIT 5").show()

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|2017|   10|   2|       Offline|             0|                     0|                             0|          65.0|       

Табличку `logs_hotel` создали, данными заполнили. 

## 2. Создание таблицы calendar

In [24]:
spark.sql("""
CREATE TABLE calendar 
USING parquet
AS
SELECT explode(
    sequence(
        to_date('2017-01-01'),
        to_date('2018-12-31'),
        interval 1 day
    )
) AS calendar_dt
""")

DataFrame[]

Проверка:

In [25]:
spark.sql("SELECT * FROM calendar LIMIT 5").show()
spark.sql("SELECT count(*) FROM calendar").show()

+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
+-----------+

+--------+
|count(1)|
+--------+
|     730|
+--------+



calendar создалась, всего 730 записей.

## 3. Выполнение запросов

### Задание 1

> Вычислить среднее количество ночей, которые гости проводят в отеле (только для подтвержденных бронирований, с детализацией по месяцам и годам)

In [37]:
spark.sql("""
SELECT
    year,
    month,
    AVG(weekend_nights + week_nights) AS avg_nights
FROM logs_hotel
WHERE status = 'Not_Canceled'
GROUP BY year, month
ORDER BY year, month
""").show()

+----+-----+------------------+
|year|month|        avg_nights|
+----+-----+------------------+
|2017|    7|3.0166666666666666|
|2017|    8|2.7189384800965017|
|2017|    9|2.6550783912747105|
|2017|   10|2.7032898820608318|
|2017|   11|2.7241935483870967|
|2017|   12| 3.043046357615894|
|2018|    1|2.7414141414141415|
|2018|    2|2.6891679748822606|
|2018|    3|3.0392038600723765|
|2018|    4| 2.924755887421022|
|2018|    5|2.8054545454545456|
|2018|    6| 2.596757322175732|
|2018|    7|3.1938088829071334|
|2018|    8|3.1544117647058822|
|2018|    9| 2.786425902864259|
|2018|   10|2.8910186199342824|
|2018|   11|2.9784511784511785|
|2018|   12|3.2521891418563924|
+----+-----+------------------+



### Задание 2

> Определить ТОП-3 месяца по проценту отмененных броней за 2018 год.

In [51]:
spark.sql("""
SELECT
    month,
    ROUND(100.0 * SUM(CASE WHEN status = 'Canceled' THEN 1 ELSE 0 END) / COUNT(*), 2) AS canceled_percent
FROM default_db.logs_hotel
WHERE year = 2018
GROUP BY month
ORDER BY canceled_percent DESC
LIMIT 3
""").show()

+-----+----------------+
|month|canceled_percent|
+-----+----------------+
|    8|           46.55|
|   10|           46.36|
|    9|           45.78|
+-----+----------------+



### Задание 3

> Вычислить среднее время на каждый месяц между бронированием и заездом в отель для подтвержденных броней.

In [55]:
spark.sql("""
SELECT
    year,
    month,
    ROUND(AVG(lead_time), 2) AS avg_lw
FROM default_db.logs_hotel
WHERE status = 'Not_Canceled'
GROUP BY year, month
ORDER BY year, month
""").show()

+----+-----+------+
|year|month|avg_lw|
+----+-----+------+
|2017|    7|130.73|
|2017|    8| 35.08|
|2017|    9| 51.72|
|2017|   10| 55.89|
|2017|   11| 33.28|
|2017|   12| 46.75|
|2018|    1| 34.87|
|2018|    2| 30.53|
|2018|    3| 43.19|
|2018|    4| 62.49|
|2018|    5| 60.99|
|2018|    6| 70.64|
|2018|    7| 86.88|
|2018|    8| 83.09|
|2018|    9| 63.32|
|2018|   10| 73.24|
|2018|   11| 44.25|
|2018|   12| 69.75|
+----+-----+------+



## Задание 4

> Вычислить общую среднюю выручку на каждый месяц в каждом году, сгруппировав по всем типам бронирования для подтвержденных броней, и вывести это в виде сводной таблицы (PIVOT).

In [58]:
spark.sql("""
SELECT *
FROM (
    SELECT
        year,
        month,
        avg_room_price
    FROM default_db.logs_hotel
    WHERE status = 'Not_Canceled'
) 
PIVOT (ROUND(AVG(avg_room_price), 2) FOR month IN (1,2,3,4,5,6,7,8,9,10,11,12))
ORDER BY year
""").show()

+----+----+-----+-----+-----+------+------+------+------+------+------+-----+-----+
|year|   1|    2|    3|    4|     5|     6|     7|     8|     9|    10|   11|   12|
+----+----+-----+-----+-----+------+------+------+------+------+------+-----+-----+
|2017|NULL| NULL| NULL| NULL|  NULL|  NULL| 77.64| 91.25|101.34| 92.94|72.25|75.42|
|2018|74.7|81.08|87.31|99.56|112.35|109.53|113.37|120.57|122.69|109.18|96.61|94.57|
+----+----+-----+-----+-----+------+------+------+------+------+------+-----+-----+



## Задание 5

> Выявить ТОП-5 постоянных гостей, которые принесли наибольшую выручку за все время, и показать их долю в общей выручке от постоянных гостей. Использовать уникальный идентификатор брони как уникальный идентификатор гостя, предположив, что 1 бронь = 1 гость.

In [84]:
spark.sql("""
WITH guest_to_profit AS (
    SELECT
        ID AS guest_id,
        SUM(avg_room_price) AS profit,
        (SELECT SUM(avg_room_price) 
            FROM default_db.logs_hotel
            WHERE status = 'Not_Canceled') as overall_revenue
    FROM default_db.logs_hotel
    WHERE status = 'Not_Canceled'
    GROUP BY ID
)
SELECT
    guest_id,
    profit,
    (100 * profit / overall_revenue) AS profit_ratio
FROM guest_to_profit
ORDER BY profit DESC
LIMIT 5
""").show()

+--------+------+--------------------+
|guest_id|profit|        profit_ratio|
+--------+------+--------------------+
|INN09462| 375.5| 0.01540622083011085|
|INN34307|349.63|0.014344812220590296|
|INN14774|332.57| 0.01364486514372827|
|INN33956| 314.1|0.012887067810220556|
|INN04151| 306.0|0.012554736548638935|
+--------+------+--------------------+



## Задание 6

> Вывести общее количество гостей на каждый день в отеле, отсортировав по убыванию дат, включая дни, когда отель пустует. Также рассчитать процент загрузки для каждого дня, если известно, что общая вместимость отеля 400 человек.

In [99]:
spark.sql("""
SELECT
    c.calendar_dt AS date,
    SUM(h.n_adults + h.n_children) AS total_guests,
    ROUND(SUM(h.n_adults + h.n_children) / 400 * 100, 2) AS occupancy_pct
FROM default_db.calendar c
LEFT JOIN default_db.logs_hotel h
    ON c.calendar_dt = to_date(concat(year, '-', lpad(month, 2, '0'), '-', lpad(date, 2, '0')), 'yyyy-MM-dd')
    AND h.status = 'Not_Canceled'
GROUP BY c.calendar_dt
ORDER BY c.calendar_dt DESC
""").show()

+----------+------------+-------------+
|      date|total_guests|occupancy_pct|
+----------+------------+-------------+
|2018-12-31|          67|        16.75|
|2018-12-30|         166|         41.5|
|2018-12-29|         162|         40.5|
|2018-12-28|         134|         33.5|
|2018-12-27|         263|        65.75|
|2018-12-26|         117|        29.25|
|2018-12-25|          84|         21.0|
|2018-12-24|          98|         24.5|
|2018-12-23|         113|        28.25|
|2018-12-22|          89|        22.25|
|2018-12-21|          91|        22.75|
|2018-12-20|          64|         16.0|
|2018-12-19|          68|         17.0|
|2018-12-18|          83|        20.75|
|2018-12-17|          82|         20.5|
|2018-12-16|         124|         31.0|
|2018-12-15|          53|        13.25|
|2018-12-14|          44|         11.0|
|2018-12-13|          54|         13.5|
|2018-12-12|          51|        12.75|
+----------+------------+-------------+
only showing top 20 rows

